# Vehicle Detection Pipeline
This notebook will process a video to detect cars in the video

In [3]:
#cell block 1
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from prod_feature_extraction import *
from prod_sliding_window import *
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
import pickle
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.models import load_model
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Using TensorFlow backend.


In [6]:
#cell block 2
#load classifier, scaler, and feature select
with open('linearsvc_vehicle_classifier.pkl', 'rb') as f:
    classifier_dict = pickle.load(f)


clf=load_model("model_alt.h5")
X_scaler = classifier_dict["scaler"]
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (24, 24) # Spatial binning dimensions
hist_bins = 20    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [500, None] # Min and max in y to search in slide_window()
def video_process_lines(img):
    image = img
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    org_image=np.copy(image)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.85, 0.8))
    
   
    
    #windows.extend(slide_window(image, x_start_stop=[500, 1000], y_start_stop=[380,500], 
    #                    xy_window=(60, 60), xy_overlap=(0.65, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(80, 80), xy_overlap=(0.7, 0.7)))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(90, 90), xy_overlap=(0.60, 0.7)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(110, 110), xy_overlap=(0.50, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-230, None], y_start_stop=[380,600], 
                        xy_window=(110, 110), xy_overlap=(0.80, 0.75)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(90, 90), xy_overlap=(0.80, 0.75)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-150, None], y_start_stop=[380,700], 
                        xy_window=(150, 150), xy_overlap=(0.71, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.8, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-480, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.8, 0.8)))
    

    
    
    hot_windows = search_windows(image, windows, clf, X_scaler,model, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

    
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(org_image, labels)


    
    return draw_img
    

In [7]:
#cell block 3
# Process video

white_output = 'deep_processed.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1=clip1.subclip(15,17)
white_clip = clip1.fl_image(video_process_lines) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video deep_processed.mp4
[MoviePy] Writing video deep_processed.mp4


100%|█████████▉| 1260/1261 [1:07:46<00:03,  3.19s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: deep_processed.mp4 

CPU times: user 1h 10min 42s, sys: 1min 49s, total: 1h 12min 32s
Wall time: 1h 7min 47s
